VAMOS IMPORTAR AS BIBLIOTECAS GLOB QUE VAI BUSCAR DETERMINADO ARQUIVO
DATETIME QUE VAI AJUDAR COM OS LOGS E XML.ELEMENTTREE QUE VAI TRABALHAR
COM ARQUIVO XML

In [3]:
import pandas as pd
import glob #! este módulo ajuda na seleção de arquivos
import xml.etree.ElementTree as ET #! este módulo ajuda no processamento de arquivos XML
from datetime import datetime


NOMEAMOS OS NOMES DOS ARQUIVOS QUE SERÃO CRIADOS

In [4]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored


VAMOS CRIAR UMA FUNÇÃO STANDARD DE LEITURA DE ARQUIVO CSV QUE VAMOS USAR
COMO PARTE DE OUTRA FUNÇÃO

In [5]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe
    

VAMOS CRIAR UMA FUNÇÃO STANDARD DE LEITURA DE ARQUIVO JSON QUE VAMOS USAR
COMO PARTE DE OUTRA FUNÇÃO

In [6]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

VAMOS CRIAR UMA FUNÇÃO STANDARD DE LEITURA DE ARQUIVO XML QUE VAMOS USAR
COMO PARTE DE OUTRA FUNÇÃO

In [7]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index=True)
    return dataframe

VAMOS CRIAR UM DATA FRAME VAZIO COM AS COLUNAS NAME, HEIGHT E WEIGHT E VAMOS USAR AS FUNÇÕES QUE CRIAMOS 
ACIMA PARA EXTRAIR OS ARQUIVOS E ISERIR NO DATA FRAME VAZIO CHAMADO EXTRACTED_DATA

In [8]:
def extract():
    extracted_data = pd.DataFrame(columns=['name','height','weight'])
    
    #! glob vai retornar uma lista com todos docs tipo csv no diretório
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #! glob vai criar uma lista com todos docs tipo json no diretório
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #! glob vai criar uma lista com todos docs tipo xml no diretório
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data
   

VAMOS CRIAR UMA FUNÇÃO STANDART PARA USAR MAIS TARDE QUE VAI TRANSFORMAR AS MEDIDAS 

In [9]:
def transform(data):
        # converte de polegadas para metros
        data['height'] = round(data.height * 0.0254,2)
        
        
        # converte de libras para kilogramas
        data['weight'] = round(data.weight * 0.45359237,2)
        return data
      


VAMOS CRIAR UMA FUNÇÃO STANDARD PARA CRIAR UM ARQUIVO CSV EM NOSSO 
DIRETÓRIO COM AS INFORMAÇÕES DAS TRANSFORMAÇÕES AUTOMATICAMENTE

In [10]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

VAMOS CRIAR OS LOGS DE INFORMAÇÃO DE CADA PROCESSO DO ETL

In [11]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')    



LOG DE INICIO DO PROCESSO ETL

In [12]:
log("ETL Job Started")

VAMOS COMEÇAR A EXTRAÇÃO DOS DADOS

In [13]:
log("Extract phase Started")
extracted_data = extract()

/tmp/ipykernel_10104/3636249690.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
/tmp/ipykernel_10104/3636249690.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
/tmp/ipykernel_10104/3636249690.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
/tmp/ipykernel_10104/3636249690.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_c

INFORMAÇÃO DE EXTRÇÃO ATRAVÉS DO LOG

In [14]:
log("Extract phase Ended")
extracted_data

,name,height,weight,Unnamed: 0.1,Unnamed: 0
0,alex,65.78,112.99,NaN,NaN
1,ajay,71.52,136.49,NaN,NaN
2,alice,69.40,153.03,NaN,NaN
3,ravi,68.22,142.34,NaN,NaN
4,joe,67.79,144.30,NaN,NaN
...,...,...,...,...,...
112,ivan,67.62,114.14,NaN,NaN
113,simon,67.90,112.37,NaN,NaN
114,jacob,66.78,120.67,NaN,NaN
115,cindy,66.49,127.45,NaN,NaN


TRANSFORMANDO OS DADOS DE HEIGHT E WEIGHT

In [15]:
log("Transform phase Started")
transformed_data = transform(extracted_data)


INFORMAÇÃO DE LOG DA TRANSFORMAÇÃO DOS DADOS

In [16]:
log("Transform phase Ended")
transformed_data 

,name,height,weight,Unnamed: 0.1,Unnamed: 0
0,alex,1.67,51.25,NaN,NaN
1,ajay,1.82,61.91,NaN,NaN
2,alice,1.76,69.41,NaN,NaN
3,ravi,1.73,64.56,NaN,NaN
4,joe,1.72,65.45,NaN,NaN
...,...,...,...,...,...
112,ivan,1.72,51.77,NaN,NaN
113,simon,1.72,50.97,NaN,NaN
114,jacob,1.70,54.73,NaN,NaN
115,cindy,1.69,57.81,NaN,NaN


VAI DAR INFORMAÇÃO DE LOG E VAI CRIAR O ARQUIVO CSV
COM OS DADOS TRANSFORMADOS E CARREGADOS

In [17]:
log("Load phase Started")
load(targetfile,transformed_data)


INFORMAÇÃO DE LOG PARA FINALIZAÇÃO DO ETL

In [18]:
log("Load phase Ended")
log("ETL Job Ended")